# 도서자료 Text Summarization

## 0) Import

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from sklearn.model_selection import train_test_split

from konlpy.tag import Komoran

## 1) AI Hub .json file to .csv file

AI Hub의 도서자료 파일은 각각의 내용이 하나하나 json파일로 이루어져있습니다.  
이를 통합하여 데이터프레임을 만들고 파일로 저장하는 코드입니다.  
매우 시간이 오래 걸리니 변환한 csv파일을 사용하면 좋을 것 같습니다.

In [2]:
"""
train_cnt = 0
test_cnt = 0
train_data_dir = "../../dataset/BookSummarization/train/"
test_data_dir = "../../dataset/BookSummarization/test/"

for path in os.listdir(train_data_dir):
    if os.path.isfile(os.path.join(train_data_dir, path)):
        train_cnt += 1

for path in os.listdir(test_data_dir):
    if os.path.isfile(os.path.join(test_data_dir, path)):
        test_cnt += 1

print(f'학습 데이터 개수 = {train_cnt}, 테스트 데이터 개수 = {test_cnt}')
"""

학습 데이터 개수 = 160002, 테스트 데이터 개수 = 19999


In [3]:
"""
test_passage_data = []
test_summary_data = []

for i in range(len(os.listdir(test_data_dir))):
    target = pd.read_json(test_data_dir+os.listdir(test_data_dir)[i])
    test_passage_data.append(target['passage'][0])
    test_summary_data.append(target['summary'][0])
    
test_data = {
    'passage': test_passage_data, 'summary': test_summary_data
}

test_df = pd.DataFrame(test_data)

train_passage_data = []
train_summary_data = []

for i in range(len(os.listdir(train_data_dir))):
    target = pd.read_json(train_data_dir+os.listdir(train_data_dir)[i])
    train_passage_data.append(target['passage'][0])
    train_summary_data.append(target['summary'][0])
    
train_data = {
    'passage': train_passage_data, 'summary': train_summary_data
}

train_df = pd.DataFrame(train_data)

train_df.to_csv("../../dataset/BookSummarization/train_df/Book_train.csv")
test_df.to_csv("../../dataset/BookSummarization/test_df/Book_test.csv")
"""

## 2) Load data and preprocess

In [10]:
train = pd.read_csv('../../dataset/BookSummarization/train_df/Book_train.csv')
test = pd.read_csv('../../dataset/BookSummarization/test_df/Book_test.csv')

In [12]:
train.head()

,Unnamed: 0,passage,summary
0,0,"둘째, 제 16차 당 대회에서 공산당은 삼개대표 이론에 입각하여 생산력발전과 사회...",공산당은 16차 당대회에서 선진세력의 입당을 공식 허용했다. 이는 “당 건설 강화...
1,1,중앙기율검사위원회가 제출한 “기율보고”도 전과 크게 다른 점이 없었다. 청렴한 정...,"16차 당 대회에서는 청렴한 정치풍토 수립, 반부패투쟁 강력 추진, 법제건설 강화 ..."
2,2,"한편, 중국정치를 연구하는 학자에게 ‘제3세대’ 지도자에서 ‘제4세대’지도자로의 ...",사회주의 정치체제는 정치지도자를 선발하는 제도적 장치가 제대로 안 갖추어져 있기 ...
3,3,2) 국가정책의 변화\n 우리는 전국인대 대표들이 심의한 국무원의 “정부업무보고”...,"정부업무보고 등은 중국정부의 정책변화, 총리와 주요정책임자의 기자회견은 중국정부의..."
4,4,중국 정부의 2003년 예산도 이와 같은 정책방침에 맞추어 편성되었다. 우선 정부가...,"중국 정부의 핵심지출 항목순서는 농촌경제 발전지출, 사회보장 지출 등으로 이어진다...."


In [13]:
test.head()

,Unnamed: 0,passage,summary
0,0,이 장은 후진타오(호금도) 시대가 어떤 과정을 통해 등장했는가를 검토하는 것이다....,후진타오 시대는 제 3세대 지도자가 물러나며 제 4세대 지도자가 새로운 통치 집단...
1,1,한편 사영기업가의 입당허용이 단기간에 중국정치에 변화를 몰고 올 것 같지는 않다....,사영기업가 계층은 국가의 통제를 아직 벗어나지 못했다는 등의 이유로 단기간에 중국...
2,2,그런데 제 16차 당 대회에서는 민족주의를 강조하는 경향이 이전보다 더욱 두드러졌...,"공산당이 국민정당에 대해 민족정당이 된 것, 정치보고에 민족정신 등에 대한 강조가..."
3,3,2.3.2 제10기 전국인민대표대회 제 1차 회의(2003년)\n 2003년 3월 ...,2003년의 10기 전국인대 1차 회의는 주요 국가지도자에 대한 인선이 있었고 공...
4,4,1) 인선에 대한 총괄평가\n 제 10기 전국인대 제 1차 회의에서 선출(비준)된...,10기 전국인대 1차 회의는 완전하고 실질적인 권력이양이 이루어지지 않았다. 이에...
